In [54]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from src import SeedZoneController

import datetime as dt

# Make New Cluster

In [56]:
sz = SeedZoneController()

sz.make_new_cluster()
sz.save()

ECV 52%
ECV 56%
ECV 57%
ECV 58%
ECV 58%
ECV 58%
ECV 59%
ECV 59%
ECV 59%
ECV 59%
ECV 59%
ECV 59%
sorting okay :)
Save Okay :)
